<a href="https://colab.research.google.com/github/charleslbryant/ai-research/blob/main/ludwig_llama2_7b_finetuning_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2-7b Fine-Tuning with 4bit Quantization

In the Colab menu bar, choose Runtime > Change Runtime Type and choose GPU under Hardware Accelerator.

## Install Ludwig

We'll use the latest version of Ludwig which includes support for quantized fine-tuning.

In [1]:
!pip uninstall -y tensorflow --quiet
!pip install "ludwig[llm]" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 992.5/992.5 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirement

## Set your HuggingFace API Token

Obtain a [HuggingFace API Token](https://huggingface.co/docs/hub/security-tokens) and request access to [Llama2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) before proceeding.

In [2]:
import os

from google.colab import drive
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive is mounted!")
else:
    print("Google Drive is already mounted!")

# Read the token file
with open('/content/drive/MyDrive/huggingface_ludwig_env_vars.sh', 'r') as file:
    lines = file.readlines()

# Extract the token value
token = None
for line in lines:
    if "HUGGING_FACE_LUDWIG_TOKEN" in line:
        token = line.split('=')[1].strip().strip("'")

# Set the environment variable
if token:
    os.environ["HUGGING_FACE_HUB_TOKEN"] = token
    print("Hugging Face Hub Token is set!")
else:
    print("Token not found!")

Mounted at /content/drive
Google Drive is mounted!
Hugging Face Hub Token is set!


## Configure Model Training

The Ludwig [configuration](https://ludwig.ai/latest/configuration/) specifies the components of the training job including:

- Model type (LLM) and base pretrained model name from HuggingFace
- Input and output features from the training dataset
- Quantization (4bit) and parameter-efficient fine-tuning (LoRA)
- Training hyperparameters (learning rate, batch size, etc.)
- Preprocessing (e.g., sampling to speed up training)
- Backend for execution (local, but could also be Ray)

In [3]:
import yaml

config_str = """
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

quantization:
  bits: 4

adapter:
  type: lora

prompt:
  template: |
    ### Instruction:
    {instruction}

    ### Input:
    {input}

    ### Response:

input_features:
  - name: prompt
    type: text

output_features:
  - name: output
    type: text

trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.1
"""

config = yaml.safe_load(config_str)

## Train!

Start training on your local GPU and monitor progress (including metrics) inline.

In this example, we'll be training on the [Alpaca](https://crfm.stanford.edu/2023/03/13/alpaca.html) dataset to turn Llama2-7b into a rudimentary chatbot. But you can use any dataset to fine-tune for other tasks.

In [4]:
import logging
from ludwig.api import LudwigModel


model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")
print(results)

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'None': 204 (without start and stop symbols)
INFO:ludwig.features.text_feature:Setting max length using dataset: 206 (including start and stop symbols)
INFO:ludwig.features.text_feature:max sequence length is 206 for feature 'None'
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'output': 758 (without start and stop symbols)
INFO:ludwig.features.text_feature:Setting max length using dataset: 760 (including start and stop symbols)
INFO:ludwig.featur

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Done.
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
INFO:ludwig.models.llm:==================================================
INFO:ludwig.models.llm:Trainable Parameter Summary For Fine-Tuning
INFO:ludwig.models.llm:Fine-tuning with adapter: lora
INFO:ludwig.models.llm:==================================================


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


INFO:ludwig.trainers.trainer:Tuning batch size...
INFO:ludwig.utils.batch_size_tuner:Tuning batch size...
INFO:ludwig.utils.batch_size_tuner:Exploring batch_size=1
INFO:ludwig.utils.batch_size_tuner:Throughput at batch_size=1: 0.94279 samples/s
INFO:ludwig.utils.batch_size_tuner:Exploring batch_size=2
INFO:ludwig.utils.batch_size_tuner:Throughput at batch_size=2: 1.02839 samples/s
INFO:ludwig.utils.batch_size_tuner:Exploring batch_size=4
INFO:ludwig.utils.batch_size_tuner:Throughput at batch_size=4: 1.06886 samples/s
INFO:ludwig.utils.batch_size_tuner:Exploring batch_size=8
INFO:ludwig.utils.batch_size_tuner:OOM at batch_size=8
INFO:ludwig.utils.batch_size_tuner:Selected batch_size=4
INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ TRAINING │
INFO:ludwig.utils.print_utils:╘══════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.trainers.trainer:Creating fresh model training run.
INFO:ludwig.trainers.trainer:Training for 10920 step

Training:  18%|█▊        | 1935/10920 [18:52<1:18:49,  1.90it/s, loss=0.081]

OutOfMemoryError: ignored

## What's next?

Now you can save / export your model to HuggingFace or explore [Predibase](https://predibase.com/) for serving and managed infrastructure for training larger LLMs and other model types.